In [ ]:
pip install trimesh -quite

In [ ]:
pip install git+https://github.com/CadQuery/cadquery.git -quite

In [ ]:
pip install reportlab -quite

In [ ]:
pip install python-occ -quite

In [ ]:
# Documentation Related Libraries
import os
import glob
import numpy as np
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.utils import simpleSplit
import pandas as pd

In [ ]:
# Feature Extraction Libraries
import cadquery as cq
from cadquery import exporters
import trimesh as tm
import json

In [ ]:
# OCC Related Libraries
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopoDS import topods
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.GeomAbs import GeomAbs_Line, GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Hyperbola, GeomAbs_Parabola, GeomAbs_BSplineCurve, GeomAbs_BezierCurve
from OCC.Core.TopAbs import TopAbs_EDGE
from collections import defaultdict
from OCC.Extend.DataExchange import read_step_file
from OCC.Core.GProp import GProp_GProps
from OCC.Core.Interface import Interface_Static

In [ ]:
# Initialise pdf storage folder

# The path under directory_PDF is the path to save the generated PDF.
directory_PDF = "D:\VSCODE\paper_code\PDF"

# Build a path pattern that searches all files
all_files_pattern_PDF = os.path.join(directory_PDF, "*")

# Get a list of all files in a directory
all_files_PDF = glob.glob(all_files_pattern_PDF)

for file in all_files_PDF:
    try:
        os.remove(file)
        print(f"Deleted documents: {file}")
    except Exception as e:
        print(f"Error when deleting a file: {file}, error: {e}")


# Initialise edges step storage folder
# directory_edges is the address where the generated step files are stored locally
directory_edges = "D:\VSCODE\paper_code\edges_step"

# Build a path pattern that searches all files
all_files_pattern_edges = os.path.join(directory_edges, "*")

# Get a list of all files in a directory
all_files_edges = glob.glob(all_files_pattern_edges)

for file in all_files_edges:
    try:
        os.remove(file)
        print(f"Deleted documents: {file}")
    except Exception as e:
        print(f"Error when deleting a file: {file}, error: {e}")


In [ ]:
def analyze_holes(shape, tolerance=1e-7):
    """
    Analysing rounded edges in shapes and calculating their diameters and quantities
    
    parameters:
    shape - Shape to be analysed
    
    return:
    circle_data - DataFrame containing the diameter
    circle_count - Number of rounded edges
    """
    edge_explorer = TopExp_Explorer(shape, TopAbs_EDGE)
    circular_edges_by_center = defaultdict(list)

    while edge_explorer.More():
        edge = topods.Edge(edge_explorer.Current())
        curve_adaptor = BRepAdaptor_Curve(edge)
        
        if curve_adaptor.GetType() == GeomAbs_Circle:
            circ = curve_adaptor.Circle()
            radius = circ.Radius()
            center = circ.Location()
            p1 = curve_adaptor.Value(curve_adaptor.FirstParameter())
            p2 = curve_adaptor.Value(curve_adaptor.LastParameter())
            if p1.IsEqual(p2, 1e-7):  # Check that the sides are closed
                circular_edges_by_center[(center.X(), center.Y(), center.Z())].append(radius)
        
        edge_explorer.Next()

    # Filter out the smallest diameter circle at each centre
    unique_circular_edges = []
    unique_centers = []
    for center, radii in circular_edges_by_center.items():
        min_radius = min(radii)
        unique_circular_edges.append(min_radius)
        unique_centers.append(center)

    # Check coaxial condition and filter
    def is_collinear(c1, c2, tol):
        # Check that two circle centres have the same coordinates and only one is different (tolerate errors)
        count = sum(1 for a, b in zip(c1, c2) if abs(a - b) <= tol)
        return count == 2
    
    collinear_groups = defaultdict(list)
    for i, center1 in enumerate(unique_centers):
        for j, center2 in enumerate(unique_centers):
            if i != j and is_collinear(center1, center2, tolerance):
                collinear_groups[i].append(center1)
                collinear_groups[i].append(center2)

    # Delete the middle centre of an odd number of coaxial circles
    for key in collinear_groups.keys():
        group = list(set(collinear_groups[key]))  # weight removal
        if len(group) % 2 != 0:
            group.sort()  # Sorting to remove intermediate values
            middle_index = len(group) // 2
            group.pop(middle_index)  # Delete the centre of the middle circle
        collinear_groups[key] = group

    valid_centers = set()
    for centers in collinear_groups.values():
        valid_centers.update(centers)

    filtered_edges = []
    filtered_centers = []
    for center, radius in zip(unique_centers, unique_circular_edges):
        if center in valid_centers:
            filtered_edges.append(radius)
            filtered_centers.append(center)

    # Calculate the number and diameter of circles
    circle_count = len(filtered_edges) // 2  # Calculate the number of circles and round to the nearest whole number
    circle_diameters = [2 * radius for radius in filtered_edges]

    circle_data = pd.DataFrame({'Diameter': circle_diameters})

    return circle_data, circle_count, filtered_edges, filtered_centers

In [ ]:
def group_collinear_centers(filtered_centers, tolerance=1e-7):
    """
    Divide the coaxial centres into groups of up to two centres each.
    
    parameters:
    filtered_centers - List of centres to be grouped
    tolerance - Tolerance error in determining coaxial
    
    return:
    collinear_groups - Grouping of coaxial centres, each group containing up to two centres
    """
    def is_collinear(c1, c2, tol):
        # Check that two circle centres have the same coordinates and only one is different (tolerate errors)
        count = sum(1 for a, b in zip(c1, c2) if abs(a - b) <= tol)
        return count == 2

    remaining_centers = set(filtered_centers)
    collinear_groups = []
    
    while remaining_centers:
        center1 = remaining_centers.pop()
        group = [center1]
        
        for center2 in list(remaining_centers):
            if is_collinear(center1, center2, tolerance):
                group.append(center2)
                remaining_centers.remove(center2)
                break  # Exit the inner loop when you find a coaxial centre.
        
        if len(group) > 1:  # Only groups containing two circle centres are added to the result
            collinear_groups.append(group)
    
    return collinear_groups

def calculate_axis_directions_and_relationships(collinear_groups, tolerance=1e-7):
    """
    Calculate the direction of the axes of each set of coaxial 
    centres and detect the parallel and perpendicular relationship between the axes
    
    parameters:
    collinear_groups - List of coaxial centres for each group
    tolerance - Tolerance errors in determining parallelism and perpendicularity
    
    return:
    axis_directions - List of axis directions for each set of coaxial centres
    parallel_count - Number of parallel axes
    perpendicular_count - Number of vertical axes
    """
    axis_directions = []
    parallel_count = 0
    perpendicular_count = 0

    # Calculate the axis direction for each group
    for group in collinear_groups:
        if len(group) == 2:  # Make sure each group has two centres of the circle
            center1, center2 = group
            direction_vector = np.array(center2) - np.array(center1)
            norm = np.linalg.norm(direction_vector)
            if norm != 0:
                direction_vector /= norm  # Normalised direction vector
            axis_directions.append(direction_vector)
    
    # Detecting parallel and perpendicular relationships between axes
    i = 0
    while i < len(axis_directions):
        direction_i = axis_directions[i]
        detected = False
        for j in range(i + 1, len(axis_directions)):
            direction_j = axis_directions[j]
            dot_product = np.dot(direction_i, direction_j)
            if np.abs(dot_product - 1) <= tolerance or np.abs(dot_product + 1) <= tolerance:
                parallel_count += 1
                detected = True
                break
            elif np.abs(dot_product) <= tolerance:
                perpendicular_count += 1
                detected = True
                break
        if detected:
            # Once a parallel or perpendicular relationship is detected, the current axis is removed and the test is repeated from the beginning.
            axis_directions.pop(i)
        else:
            i += 1
    
    return axis_directions, parallel_count, perpendicular_count

In [ ]:
def classify_edges_in_shape(shape):
    """
    Classify the edges in a shape as straight and curved and count the number of edges of each type
    
    parameters:
    shape - Shape to be analysed
    
    return:
    straight_edge_count - Number of straight edges
    curved_edge_count - Number of curved edges
    """
    straight_edge_count = 0
    curved_edge_count = 0

    # Iterate over all edges of the model
    edge_explorer = TopExp_Explorer(shape, TopAbs_EDGE)
    while edge_explorer.More():
        edge = topods.Edge(edge_explorer.Current())
        curve_adaptor = BRepAdaptor_Curve(edge)
        
        # Classification according to geometric type
        curve_type = curve_adaptor.GetType()
        if curve_type == GeomAbs_Line:
            straight_edge_count += 1
        else:
            curved_edge_count += 1
        
        edge_explorer.Next()
    
    straight_edge_per = straight_edge_count/(straight_edge_count + curved_edge_count) *100
    curved_edge_per = curved_edge_count/(straight_edge_count + curved_edge_count) *100

    return straight_edge_per, curved_edge_per

In [ ]:
def determine_symmetry(mesh):
    """
    Determine the symmetry description and symmetry axis/point based on the mesh's symmetry type.

    Parameters:
    mesh (object): The mesh object containing symmetry information.

    Returns:
    tuple: A tuple containing symmetry description (str) and symmetry axis/point (str or None).
    """
    symmetry = mesh.symmetry
    
    if symmetry is None:
        symmetry_description = "This model_{first_8_chars} has no symmetry."
        symmetry_axis = "None"
    elif symmetry == "radial":
        symmetry_description = "This model_{first_8_chars} is radially symmetrical."
        symmetry_axis = mesh.symmetry_axis
    elif symmetry == "spherical":
        symmetry_description = "This model_{first_8_chars} is spherically symmetrical."
        symmetry_axis = mesh.symmetry_axis
    else:
        symmetry_description = "Unknown symmetry type."
        symmetry_axis = "Unknown"
    
    return symmetry_description, symmetry_axis

In [ ]:
# CAD model design history information extraction
def parse_json_to_sequence(json_file, model_name):
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    entities = data['entities']
    sequence = data['sequence']
    
    commands = []
    descriptions = []
    
    for step in sequence:
        entity = entities[step['entity']]
        if step['type'] == 'Sketch':
            command, description = parse_sketch(entity, model_name)
        elif step['type'] == 'ExtrudeFeature':
            command, description = parse_extrude(entity, model_name)
        commands.append(command)
        descriptions.append(description)
    
    return '\n'.join(commands), '\n'.join(descriptions)

def parse_sketch(entity, model_name):
    name = entity['name']
    
    sketch_commands = [f"{name}"]
    sketch_descriptions = [f"{model_name} creates a sketch named {name} with the following features:"]
    
    for profile_key, profile_value in entity['profiles'].items():
        loops = profile_value['loops']
        for loop in loops:
            profile_curves = loop['profile_curves']
            for curve in profile_curves:
                if curve['type'] == 'Circle3D':
                    x = curve['center_point']['x']
                    y = curve['center_point']['y']
                    r = curve['radius']
                    sketch_commands.append(f"R ({x}, {y}, {r})")
                    sketch_descriptions.append(f"{model_name} creates a circle centered at ({x}, {y}) with radius {r}.")
                elif curve['type'] == 'Line3D':
                    x = curve['end_point']['x']
                    y = curve['end_point']['y']
                    sketch_commands.append(f"L ({x}, {y})")
                    sketch_descriptions.append(f"{model_name} creates a line ending at ({x}, {y}).")
                elif curve['type'] == 'Arc3D':
                    x = curve['end_point']['x']
                    y = curve['end_point']['y']
                    alpha = curve.get('sweep_angle', 0)
                    f = curve.get('is_counterclockwise', False)
                    sketch_commands.append(f"A ({x}, {y}, {alpha}, {f})")
                    direction = "counterclockwise" if f else "clockwise"
                    sketch_descriptions.append(f"{model_name} creates an arc ending at ({x}, {y}), sweeping {alpha} degrees {direction}.")
    
    return ' -> '.join(sketch_commands), '\n'.join(sketch_descriptions)

def parse_extrude(entity, model_name):
    name = entity['name']
    extent_one = entity['extent_one']
    extent_two = entity['extent_two']
    operation = entity['operation']
    
    if 'transform' in entity:
        transform = entity['transform']
        theta = transform['x_axis']['x']
        phi = transform['y_axis']['y']
        psi = transform['z_axis']['z']
        px = transform['origin']['x']
        py = transform['origin']['y']
        pz = transform['origin']['z']
    else:
        theta = phi = psi = 0
        px = py = pz = 0
    
    s = 1 
    e1 = extent_one['distance']['value']
    e2 = extent_two['distance']['value']
    b = operation
    u = entity['extent_type']
    
    command = f"E ({theta}, {phi}, {psi}, {px}, {py}, {pz}, {s}, {e1}, {e2}, {b}, {u})"
    description = f"{model_name} performs an extrude operation named {name} with the following parameters:\n" \
                  f"{model_name} sets the direction to ({theta}, {phi}, {psi}),\n" \
                  f"{model_name} sets the origin to ({px}, {py}, {pz}),\n" \
                  f"{model_name} sets the scale to {s},\n" \
                  f"{model_name} sets the extent one to {e1},\n" \
                  f"{model_name} sets the extent two to {e2},\n" \
                  f"{model_name} sets the operation to {b},\n" \
                  f"{model_name} sets the extent type to {u}."
    
    return command, description

In [ ]:
# Define a function to check if a page break is needed
def check_page_break(y, c):
    if y < 40: 
        c.showPage()
        c.setFont("Helvetica", 12)
        return 750 
    return y

In [ ]:
# Catalogue Initial

# PDF_directory is the local storage path for generated PDF files.
PDF_directory = "D:\VSCODE\paper_code\PDF"
# Build a path pattern that searches all files
all_PDFfiles_pattern = []
all_PDFfiles_pattern = os.path.join(PDF_directory, "*")

# Get a list of all files in a directory
all_PDFfiles = []
all_PDFfiles = glob.glob(all_PDFfiles_pattern)

for PDFfile in all_PDFfiles:
    try:
        os.remove(PDFfile)
        print(f"Deleted documents: {PDFfile}")
    except Exception as e:
        print(f"Error when deleting a file: {PDFfile}, error: {e}")

# Initial step file
# Output the path to the folder next to step
output_edges_folder_path = "D:\VSCODE\paper_code\edges_step"
# Build a path pattern that searches all files
all_stepfiles_pattern = []
all_stepfiles_pattern = os.path.join(output_edges_folder_path, "*")

# Get a list of all files in a directory
all_stepfiles = []
all_stepfiles = glob.glob(all_stepfiles_pattern)

for stepfile in all_stepfiles:
    try:
        os.remove(stepfile)
        print(f"Deleted documents: {stepfile}")
    except Exception as e:
        print(f"Error when deleting a file: {stepfile}, error: {e}")

# Initial stl file
# Output the path to the folder next to step
stl_path = 'D:\\VSCODE\\paper_code\\newstl'
# Build a path pattern that searches all files
all_stl_pattern = []
all_stl_pattern = os.path.join(stl_path, "*")

# Get a list of all files in a directory
all_stlfiles = []
all_stlfiles = glob.glob(all_stl_pattern)

for stlfile in all_stlfiles:
    try:
        os.remove(stlfile)
        print(f"Deleted documents: {stlfile}")
    except Exception as e:
        print(f"Error when deleting a file: {stlfile}, error: {e}")


In [ ]:
# Importing a step file

# Model folder path
folder_path = "D:\\VSCODE\\paper_code\\dataset\\ABC_parts_210000-219999\\acctest"
 # deep_data_directory is the local storage path for CAD model deepldata.
deep_data_directory = 'D:\\VSCODE\\paper_code\\deepdata\\0021'

# Iterate through all files in a folder
for file_name in os.listdir(folder_path):
    # Constructing full file paths
    model_file_path = None
    model_file_path = os.path.join(folder_path, file_name)

    # Get the filename of the currently read file
    current_file_name = []
    first_8_chars = []
    current_file_name = os.path.basename(model_file_path)
    first_8_chars = current_file_name[:8]
    
    # Importing STEP files
    model = None
    model = cq.importers.importStep(model_file_path)

    # Convert to stl file
    new_stl_name = []
    new_stl_name = first_8_chars+ ".stl"
    output_stl_file_path = None
    output_stl_file_path = os.path.join(stl_path, new_stl_name)

    cq.exporters.export(model, output_stl_file_path)
    
    shape = None
    shape = model.val()  # Getting Shape Objects

    # Get cylindrical faces and rounded edges
    circular_edges = None
    circular_edges = model.faces("%CYLINDER").edges("%CIRCLE")

    # Construct a new filename starting with edges
    new_file_name = []
    new_file_name = "edges_" + first_8_chars+ ".step"

    # Construct the full output file path
    output_edges_file_path = None
    output_edges_file_path = os.path.join(output_edges_folder_path, new_file_name)

    # Exporting processed STEP files
    cq.exporters.export(model, output_edges_file_path)

    # read step file
    step_reader = STEPControl_Reader()
    for file in os.listdir(output_edges_folder_path):
        edges_file_path = None
        edges_file_path = os.path.join(output_edges_folder_path, file)

    status = []   
    unit = None 
    status = step_reader.ReadFile(edges_file_path)

    if status == 1:  # IFSelect_RetDone == 1
        step_reader.TransferRoots()
        unit = Interface_Static.CVal("xstep.cascade.unit")
        shape = step_reader.OneShape()
    else:
        raise Exception("Failed to read the STEP file")
    

    
    # Calculate the number and diameter of holes
    circle_data = []
    circle_count = []
    filtered_edges = []
    filtered_centers = []
    circle_data, circle_count, filtered_edges, filtered_centers = analyze_holes(shape, tolerance=1e-7)

    if circle_count > 0:
        collinear_groups = None
        collinear_groups = group_collinear_centers(filtered_centers, tolerance=1e-7)

        axis_directions = []
        parallel_count = []
        perpendicular_count = []
        axis_directions, parallel_count, perpendicular_count = calculate_axis_directions_and_relationships(collinear_groups)

    # Calculate the number of straight and curved edges of the model
    whole_shape = None
    whole_shape = read_step_file(model_file_path)

    straight_edge_per = []
    curved_edge_per = []
    straight_edge_per, curved_edge_per = classify_edges_in_shape(whole_shape)

    # Computational base information
    mesh = None
    mesh = tm.load(output_stl_file_path)

    #Extract model volume
    Volume = []
    Volume = mesh.volume
    
    #Extract model surface area
    surface_area = []
    surface_area = mesh.area

    # Calculate the volume of the enclosing box
    bounding_boxmesh = None
    box_volume = []
    bounding_boxmesh = mesh.bounding_box_oriented
    box_volume = bounding_boxmesh.volume

    # Calculating Gaussian curvature
    gauss_curvature = []
    variance_gauss = []
    gauss_curvature = tm.curvature.vertex_defects(mesh)
    variance_gauss = np.var(gauss_curvature)

    mean_curvature = []
    variance_mean = []
    mean_curvature = mesh.integral_mean_curvature
    variance_mean = np.var(mean_curvature)

    # Determine if a plane
    is_flat = None
    is_flat = np.abs(gauss_curvature) < 1e-1
    # Calculate the proportions of planes and surfaces
    flat_ratio = []
    curved_ratio = []
    curved_ratio = np.sum(is_flat) / len(gauss_curvature) *100
    flat_ratio = 100 - curved_ratio

    symmetry_description = None
    symmetry_axis = None
    symmetry_description, symmetry_axis = determine_symmetry(mesh)

    # deep data
    new_json_name = []
    new_json_name = first_8_chars+ ".json"
    json_file_path = None
    json_file_path = os.path.join(deep_data_directory, new_json_name)
    json_model_name = None
    json_model_name = f"model_{first_8_chars}"
    command_sequence, descriptions = parse_json_to_sequence(json_file_path, json_model_name)

    # Generate PDF
    # Generate file name
    PDF_filename = f"model_{first_8_chars}.pdf"
    
    # Creating full file paths
    PDF_file_path = os.path.join(PDF_directory, PDF_filename)
    
    # Create a Canvas object
    c = canvas.Canvas(PDF_file_path, pagesize=letter)
    
    # Setting fonts and sizes
    c.setFont("Helvetica", 12)
    # Set the initial coordinates.
    x = 20
    y = 730
    page_height = letter[1]
    
    # Add Text
    c.drawString(20, y, f"The volume of the model_{first_8_chars} is {Volume:.3f} cubic {unit}.")
    y -= 20
    c.drawString(20, y, f"The surface area of the model_{first_8_chars} is {surface_area:.3f} square {unit}.")
    y -= 20
    c.drawString(20, y, f"The volume of this model_{first_8_chars} bounding box is {box_volume:.3f} cubic {unit}.")
    y -= 20
    c.drawString(20, y, f"The variance of the Gaussian curvature of this model_{first_8_chars} is {variance_gauss:.3f}.")
    y -= 20
    c.drawString(20, y, f"The variance of the Mean curvature of this model_{first_8_chars} is {variance_mean:.3f}.")
    y -= 20
    c.drawString(20, y, f"The flat plane of this model_{first_8_chars} occupies {flat_ratio:.3f}%.")
    y -= 20
    c.drawString(20, y, f"The curved surface of this model_{first_8_chars} occupies {curved_ratio:.3f}%.")
    y -= 20
    # Add information about straight and curved edges
    c.drawString(20, y, f"The straight edge of this model_{first_8_chars} occupies {straight_edge_per:.3f}%.")
    y -= 20
    y = check_page_break(y, c)
    c.drawString(20, y, f"The curved edge of this model_{first_8_chars} occupies {curved_edge_per:.3f}%.")
    y -= 20
    y = check_page_break(y, c)
    c.drawString(20, y, f"{symmetry_description}")
    y -= 20
    c.drawString(20, y, f"The axis of symmetry of this model_{first_8_chars} is {symmetry_axis}.")
    y -= 20

    # Adding information about the hole
    for index, row in circle_data.iterrows():
        c.drawString(20, y, f"One of the round holes in this model_{first_8_chars} has a diameter {index + 1} of {row['Diameter']:.3f} {unit}")
        y -= 20
        y = check_page_break(y, c)

    c.drawString(20, y, f"Total number of round holes of this model_{first_8_chars} is {circle_count}.")
    y -= 20

    # Adding information about the central axis
    if circle_count > 1:
        c.drawString(20, y, f"The number of all parallel holes for model_{first_8_chars} is {parallel_count}.")
        y -= 20
        y = check_page_break(y, c)
        c.drawString(20, y, f"The number of all perpendicular holes for model_{first_8_chars} is {perpendicular_count}.")
        y -= 20
        y = check_page_break(y, c)


    # Generate deepdata information.
    
    lines = simpleSplit(descriptions, c._fontname, c._fontsize, letter[0] - 40)
    for line in lines:
        c.drawString(20, y, line)
        y -= 20
        y = check_page_break(y, c)
    

    c.save()

   

    
